In [1]:
from pycocos import Cocos
import os
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from itertools import product
import time
tickers = [
    "TZX25", "TZX26", "TZX27", "TZX28", "TZXD5", "TZXD6", "TZXD7",
    "TZXM5", "TZXM6", "TZXM7", "TZXY5", "TZXO5", "TZXO6", "T5X4",
    "T2X5", "TC25P", "TX25", "TX26", "TX28", "TX31"
]
combinations = list(product(tickers, repeat=2))

In [2]:
app = Cocos(email="cirigliano.santiago@gmail.com", 
            password=os.getenv('CLAVE_SECRETA_COCOS'), 
            api_key="OPTIONAL", 
            topt_secret_key=os.getenv('topt_secret_key'))

def make_long_ticker(ticker,term,currency):
    if term == "CI": settlement = app.settlements.T0
    elif term == "24" or term =="24hs": settlement = app.settlements.T1
    if currency == "pesos": currency=app.currencies.PESOS
    elif currency == "dolares": currency=app.currencies.USD

    long_ticker = app.long_ticker(ticker=ticker, 
                            settlement=settlement, 
                            currency=currency)
    return long_ticker

def compare(tickers):
    ticker1=tickers[0]
    ticker2=tickers[1]
    tx26_=make_long_ticker(ticker1,"24","pesos")
    tx28_=make_long_ticker(ticker2,"24","pesos")
    tx26=app.get_daily_history(long_ticker= tx26_, date_from= "2024-04-19")
    tx28=app.get_daily_history(long_ticker= tx28_, date_from= "2024-04-19")
    
    # Convertir a dataframes
    df_activo_1 = pd.DataFrame(tx26["prices"])
    df_activo_2 = pd.DataFrame(tx28["prices"])

    # Unir los dataframes en base a la fecha
    df_combined = pd.merge(df_activo_1, df_activo_2, on="date", suffixes=('_1', '_2'))

    # Eliminar filas con valores None en cualquiera de los precios de cierre
    df_combined.dropna(subset=['close_1', 'close_2'], inplace=True)

    # Calcular la división de los precios de cierre
    df_combined['ratio'] = df_combined['close_1'] / df_combined['close_2']

    max_ratio = df_combined['ratio'].max()

    # Calcular qué porcentaje del máximo representa cada ratio
    df_combined['spread_pct'] = (df_combined['ratio'] / max_ratio) * 100 

    # Convertir fechas a un formato numérico para el modelo
    df_combined['date_numeric'] = np.arange(len(df_combined))

    # Ajustar el modelo lineal
    X = df_combined[['date_numeric']]
    y = df_combined['ratio']
    model = LinearRegression()
    model.fit(X, y)

    # Predicciones del modelo
    df_combined['ratio_pred'] = model.predict(X)

    # Calcular el error estándar residual y el intervalo de confianza
    confidence_level = 0.999
    z_score = norm.ppf((1 + confidence_level) / 2)

    # Calcular el error estándar de las predicciones
    residuals = y - df_combined['ratio_pred']
    se = residuals.std(ddof=1) / np.sqrt(len(df_combined))

    # Limites superior e inferior del intervalo de confianza
    df_combined['lower_bound'] = df_combined['ratio_pred'] - z_score * se
    df_combined['upper_bound'] = df_combined['ratio_pred'] + z_score * se
    spread= 100-df_combined['spread_pct'].min()

    # Crear el gráfico en Plotly
    fig = go.Figure()
    # Puntos originales
    fig.add_trace(go.Scatter(x=df_combined['date'], 
                             y=df_combined['ratio'], 
                             mode='markers', name='Ratio Original'))
    
    fig.add_trace(go.Scatter(x=df_combined['date'], 
                             y=df_combined['spread_pct'], 
                             mode='markers', name='Ratio %',yaxis='y2'))
    
    # Línea del modelo lineal
    fig.add_trace(go.Scatter(x=df_combined['date'], 
                             y=df_combined['ratio_pred'], 
                             mode='lines', name='Modelo Lineal'))

    # Banda de confianza
    fig.add_trace(go.Scatter(
        x=pd.concat([df_combined['date'], df_combined['date'][::-1]]),
        y=pd.concat([df_combined['upper_bound'], df_combined['lower_bound'][::-1]]),
        fill='toself',
        fillcolor='rgba(0,100,80,0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=True,
        name='Confianza 70%'
    ))

    fig.update_layout(
        title=f"Ratio {ticker1}/{ticker2}    spread {round(spread,0)}",
        xaxis_title="Fecha",
        yaxis_title="Ratio de precios de cierre",
        yaxis2=dict(
            title="Spread %",
            overlaying='y', 
            side='right'
        ),
        legend=dict(orientation="h", x=0.5, y=-0.2, xanchor='center', yanchor='top')
    )
    fig.show()

for ticker in [["AL30","GD30"],["TZXM6","TZXM7"],["TZX26","TZX27"],["TZXD6","TZXD7"],["TX26","TX28"]]:
    try:
        time.sleep(1)
        compare(ticker)
    except Exception as e:
        print(str(e))